### Preparation

In [1]:
#!pip install numpy
#!pip install pandas
#!pip install requests
#!pip install tqdm
#!pip install sqlalchemy

### Packages

In [2]:
import numpy as np
import pandas as pd
import requests
import os
from tqdm import tqdm
from sqlalchemy import create_engine

### Input

In [3]:
coins = ('BTCUSDT', 'ETHUSDT', 'BNBUSDT', 'SOLUSDT', 'ADAUSDT', 'DOTUSDT', 'DOGEUSDT', 'AVAXUSDT', 'SHIBUSDT',
         'MATICUSDT', 'LTCUSDT', 'UNIUSDT', 'ALGOUSDT', 'TRXUSDT', 'LINKUSDT', 'MANAUSDT', 'ATOMUSDT', 'VETUSDT',
         'FTMUSDT')

capital   = 100 #$
UTC       = 7
SMA       = 28 #Simple Moving Average
timeframe = '4h'
#tradingfee = 0

### Download data

In [4]:
def download_data(symbol, timeframe):
    
    resp = requests.get('https://api.binance.us/api/v3/klines?symbol=' + symbol + '&interval=' + timeframe + '&limit=1000')
    frame = pd.DataFrame(resp.json())
    frame = frame.iloc[:,:5]
    frame.columns = ['Time', 'Open', 'High', 'Low', 'Close']
    frame[['Open', 'High', 'Low', 'Close']] = frame[['Open', 'High', 'Low', 'Close']].astype(float)
    frame.Time = pd.to_datetime(frame.Time, unit='ms') + pd.Timedelta(hours=UTC)

    return frame

In [5]:
if os.path.exists("Crypto.db"):
    os.remove("Crypto.db")
    
engine = create_engine('sqlite:///Crypto.db')

for coin in tqdm(coins):
    download_data(coin, timeframe).to_sql(coin, engine, index=False)

100%|██████████| 19/19 [00:18<00:00,  1.05it/s]


### Main Program

In [6]:
class Backtest_SMA:
    
    def __init__(self, capital, SMA, coins): 
        self.capital = capital
        self.SMA     = SMA
        self.coins   = coins
            
    def method(self, coin):
        data = pd.read_sql(coin, engine).set_index('Time')  
        
        data['sma'] = data.Close.rolling(self.SMA).mean()
        data.dropna(inplace=True)
    
        data['Type'] = np.where(data['Close'].shift(1) > data['sma'].shift(1), 'long', 'short')
        
        data['change'] = data.Close.pct_change()
        
    
        data['long strategy']  = np.where(data['Close'] > data['sma'], 1, 0)
        data['Long positions'] = self.capital*(np.cumprod(data['long strategy'].shift(1)*data['change'] + 1))
        
        data['short strategy']  = np.where(data['Close'] > data['sma'], 0, -1)
        data['Short positions'] = self.capital*(np.cumprod(data['short strategy'].shift(1)*data['change'] + 1))
        
        data['Return SMA'] = data['Long positions']*data['Short positions']/self.capital
        data['Return hold']  = self.capital*np.cumprod(data['change']+1)
    
        data['win pos'] = np.where(data['Close']> data['sma'], 1, -1)
        data['win']     = np.cumsum(np.where(data['win pos'].shift(1)*data['change'] > 0, 1, 0))
        
        data.dropna(inplace=True)
        data['Winrate'] = data['win']/np.arange(1, len(data) + 1)
        
        return data.loc[:, ['Winrate', 'Type', 'Long positions', 'Short positions', 'Return SMA', 'Return hold']]
    
    
    def sorting(self):
        profits = []
        
        for coin in self.coins:
            df = self.method(coin)
            
            ret = df[['Return SMA', 'Return hold']]
            ret.columns = ['Return SMA ($)', 'Return hold ($)']
            
            profits.append(ret.iloc[-1,:])

        frame = pd.DataFrame(profits, self.coins)
        frame['edges ($)'] = frame['Return SMA ($)'] - frame['Return hold ($)']
        frame = frame.sort_values('edges ($)', ascending=False)
        return frame

    
    def output(self, coin):
        data = self.method(coin)
        
        print('Simulation of ' + '\033[1m' + str(coin) + '\033[0m' + ' runs from ' + '\033[1m' + str(data.index[0]) + '\033[0m' + ' to ' + '\033[1m' + str(data.index[-1]) + '\033[0m')
        print('Winrate Method: %5.2f' %(100*data['Winrate'][-1]) + '%\n')

        print('Return Hold                : $%5.2f' %(data['Return hold'][-1]))
        print('return from Long Positions : $%5.2f' %(data['Long positions'][-1]))
        print('return from Short Positions: $%5.2f' %(data['Short positions'][-1]))
        print('Return Method              : $%5.2f' %(data['Return SMA'][-1]))

        
    def journal(self, coin):
        if os.path.exists('Journal ' + coin + '.xlsx'):
            os.remove('Journal ' + coin + '.xlsx')
        
        self.method(coin).to_excel('Journal ' + coin + '.xlsx', engine='xlsxwriter') 

In [7]:
backtesting = Backtest_SMA(capital, SMA, coins)

### Output

In [8]:
#backtesting.method('AVAXUSDT')

In [9]:
backtesting.sorting()

,Return SMA ($),Return hold ($),edges ($)
AVAXUSDT,199.485788,52.948353,146.537435
ETHUSDT,200.271041,66.782997,133.488044
SOLUSDT,133.238019,34.569314,98.668705
SHIBUSDT,173.066672,84.873950,88.192722
BTCUSDT,132.718009,54.606231,78.111778
ATOMUSDT,186.350084,112.228049,74.122035
BNBUSDT,141.876587,91.410424,50.466162
DOGEUSDT,153.049283,105.801148,47.248135
ADAUSDT,74.477984,48.783938,25.694045
FTMUSDT,74.247399,50.515169,23.732230


In [10]:
backtesting.output('AVAXUSDT')

Simulation of AVAXUSDT runs from 2022-06-08 23:00:00 to 2022-11-17 19:00:00
Winrate Method: 50.00%

Return Hold                : $52.95
return from Long Positions : $118.54
return from Short Positions: $168.29
Return Method              : $199.49


In [11]:
backtesting.journal('AVAXUSDT')